In [8]:
from MySQL_DB_connect_functions import get_db_info, connect_to_db
import pandas as pd

In [9]:
host_name, dbname, schema_name, port, username, password = get_db_info()

In [10]:
cnx = None

In [11]:
cnx = connect_to_db(username, password, host_name, schema_name, port)

Connected!


In [12]:
cursor = cnx.cursor()

In [13]:
cursor.execute("SELECT * FROM videos LIMIT 5")

In [14]:
for i in cursor:
    print(i)

('_-0J49_9lwc', 'LastWeekTonight', 'Housing Discrimination: Last Week Tonight with John Oliver (HBO)', 'John Oliver breaks down the long history of housing discrimination in the U.S., the damage it’s done, and, crucially, what we can do about it. \n\nConnect with Last Week Tonight online... \n\nSubscribe to the Last Week Tonight YouTube channel for more almost news as it almost happens: www.youtube.com/lastweektonight \n\nFind Last Week Tonight on Facebook like your mom would: www.facebook.com/lastweektonight \n\nFollow us on Twitter for news about jokes and jokes about news: www.twitter.com/lastweektonight \n\nVisit our official site for all that other stuff at once: www.hbo.com/lastweektonight', datetime.date(2021, 7, 26), 6885028, 130544, 0, 16250, 1, 'Monday', 1928, 0)
('_066dEkycr4', 'LastWeekTonight', 'Coronavirus II: Last Week Tonight with John Oliver (HBO)', 'As the spread of coronavirus worsens in the US, John Oliver discusses how the government is handling the outbreak, how t